In [1]:
import igraph 
import csv
import unicodedata

import numpy
from scipy.signal import argrelextrema

In [1]:
def CONGAYearAlgorithm(graph,result, Overlap,mutual_friends, difference_gap=0):
    
    #Prepare parameters to modularity tests
    startedGraph = graph.copy()
    modularityResult = []
    
    while(graph.ecount()>0):
        #count betweenness values
        edge_betweenness = graph.edge_betweenness(directed=False, cutoff=None, weights=None)
        max_eb = max(edge_betweenness)
        vertex_betweenness = graph.betweenness(vertices=None, directed=False, cutoff=None, weights=None, nobigint=True)
        max_vb = max(vertex_betweenness)
        
        if(max_eb + difference_gap>=max_vb):
            #edge betweenness in this step
            edgesToDelete = [graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
            result.append(edgesToDelete)
            graph.delete_edges(edgesToDelete)
        else:
            #vertex betweenness in this step
            vertexToCopy = vertex_betweenness.index(max_vb);
            incident = graph.incident(vertexToCopy, mode=ALL)
            
            #check if incident edge set has more than 1 edge
            if len(incident) == 1:
                edgesToDelete = [graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
                graph.delete_edges(edgesToDelete)
                tmp = countModularity(startedGraph,graph,Overlap)
                modularityResult.append(tmp)
                continue
                
            graph.add_vertices(1)
            graph.vs[graph.vcount()-1]['name'] = graph.vs[vertexToCopy]['name']
            Overlap[graph.vcount()-1] = vertexToCopy
            
            #count average (diffrent despite of the parameter)
            sum = 0
            weight_counter = 0
            for edge in incident:
                if mutual_friends:
                    sum += (graph.es[edge]['weight']*graph.es[edge]['friends'])
                    weight_counter += graph.es[edge]['friends']
                else:
                    sum += graph.es[edge]['weight']
                    weight_counter += 1
            average = sum/weight_counter

            edges_to_delete = []
            new_vertex_counter = graph.vcount()-1
            allSameWeight = 1
            popWeight = -1
            
            for edge in incident:
                weight = graph.es[edge]['weight']
                friends = graph.es[edge]['friends']
                
                if popWeight != weight and edge != incident[0]:
                    allSameWeight = 0
                
                #select if edge go with the copy of the vertex
                if(weight<average):
                    edges_to_delete.append(edge)
                    source = graph.es[edge].source
                    target = graph.es[edge].target

                    if(target == vertexToCopy):
                        graph.add_edges([(source, new_vertex_counter)])
                    else:
                        graph.add_edges([(new_vertex_counter, target)])
                    graph.es[graph.ecount()-1]["weight"]=weight
                    graph.es[graph.ecount()-1]["friends"]=friends
                popWeight = weight
            
            #edge case with all incident edges with same width - copy of the vertex changes nothing => infinity loop
            if allSameWeight == 1:
                source = graph.es[incident[0]].source
                target = graph.es[incident[0]].target
                edges_to_delete.append(incident[0])
                
                if vertexToCopy == target:
                    graph.add_edges([(graph.es[incident[0]].source, new_vertex_counter)])
                else:
                    graph.add_edges([(new_vertex_counter,graph.es[incident[0]].target)])
                graph.es[graph.ecount()-1]["weight"]=weight
                graph.es[graph.ecount()-1]["friends"]=friends
            
            #add edges to result dendrogram
            toAdd = [(vertexToCopy, vertexToCopy)]
            for edge in edges_to_delete:
                toAdd.append((graph.es[edge].source,graph.es[edge].target))
            result.append(toAdd)
            graph.delete_edges(edges_to_delete)
        
        #count modularity from OCDLCE
        tmp = countModularity(startedGraph,graph,Overlap)
        modularityResult.append(tmp)
        
    return modularityResult

In [2]:
def CONGAYearAlgorithmPreprocessing(graph,mutual_friends):
    
    #almost same function as above, but we count difference between max edge betweenness and vertex betweenness to limit copy of the vecor
    startedGraph = graph.copy()
    difference = []
    
    while(graph.ecount()>0):
        edge_betweenness = graph.edge_betweenness(directed=False, cutoff=None, weights=None)
        max_eb = max(edge_betweenness)
        vertex_betweenness = graph.betweenness(vertices=None, directed=False, cutoff=None, weights=None, nobigint=True)
        max_vb = max(vertex_betweenness)
        
        if(max_eb>=max_vb):
            edgesToDelete = [graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
            graph.delete_edges(edgesToDelete)
        else:
            difference.append(max_vb - max_eb)      
            vertexToCopy = vertex_betweenness.index(max_vb);

            incident = graph.incident(vertexToCopy, mode=ALL)
            
            if len(incident) == 1:
                edgesToDelete = [graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
                graph.delete_edges(edgesToDelete)
                continue
                
            graph.add_vertices(1)
            graph.vs[graph.vcount()-1]['name'] = graph.vs[vertexToCopy]['name']
            
            sum = 0
            weight_counter = 0
            for edge in incident:
                if mutual_friends:
                    sum += (graph.es[edge]['weight']*graph.es[edge]['friends'])
                    weight_counter += graph.es[edge]['friends']
                else:
                    sum += graph.es[edge]['weight']
                    weight_counter += 1

            average = sum/weight_counter

            edges_to_delete = []
            new_vertex_counter = graph.vcount()-1
            allSameWeight = 1
            popWeight = -1
            
            for edge in incident:
                weight = graph.es[edge]['weight']
                friends = graph.es[edge]['friends']
                if popWeight != weight and edge != incident[0]:
                    allSameWeight = 0
                if(weight<average):
                    edges_to_delete.append(edge)
                    source = graph.es[edge].source
                    target = graph.es[edge].target

                    if(target == vertexToCopy):
                        graph.add_edges([(source, new_vertex_counter)])
                    else:
                        graph.add_edges([(new_vertex_counter, target)])
                    graph.es[graph.ecount()-1]["weight"]=weight
                    graph.es[graph.ecount()-1]["friends"]=friends
                popWeight = weight
            
            if allSameWeight == 1:
                source = graph.es[incident[0]].source
                target = graph.es[incident[0]].target
                edges_to_delete.append(incident[0])

                if vertexToCopy == target:
                    graph.add_edges([(graph.es[incident[0]].source, new_vertex_counter)])
                else:
                    graph.add_edges([(new_vertex_counter,graph.es[incident[0]].target)])
                graph.es[graph.ecount()-1]["weight"]=weight
                graph.es[graph.ecount()-1]["friends"]=friends
                        
            toAdd = [(vertexToCopy, vertexToCopy)]
            for edge in edges_to_delete:
                toAdd.append((graph.es[edge].source,graph.es[edge].target))
            graph.delete_edges(edges_to_delete)
        
    return difference

In [2]:
def countModularity(startedGraph,graph, Overlap):
    modularityGraph = startedGraph.copy()
    while modularityGraph.ecount() > 0:
        for edge in modularityGraph.es:
            modularityGraph.delete_edges(edge)
        
    edges_to_add = []
    for edge in graph.es:
        source = edge.source
        target = edge.target
        if(source == target):
            continue
        if(source>=modularityGraph.vcount()):
            while(Overlap[source]!=-1):
                source=Overlap[source]
        if(target>=modularityGraph.vcount()):
            while(Overlap[target]!=-1):
                target=Overlap[target]
        edges_to_add.append((source,target))

    modularityGraph.add_edges(edges_to_add)
    
    m = 0
    for component in graph.components():
        community = []
        for v in component:
            if(v>=modularityGraph.vcount()):
                while(Overlap[v]!=-1):
                    v=Overlap[v]
            community.append(v)
        m += modularity(modularityGraph, community)

    return m/len(graph.components())

In [ ]:
def modularity(graph, community):
    M_in = 0
    M_out = 0
    for node in community:
        for edge_ind in graph.incident(node):
            edge = graph.es[edge_ind]
            if node == edge.target and node == edge.source:
                continue
            elif node == edge.target:
                if edge.source in community:
                    M_in+=1
                else:
                    M_out+=1
            elif node  == edge.source:
                if edge.target in community:
                    M_in+=1
                else:
                    M_out+=1
    if M_out == 0:
        #change it from the original OCDCLE modularity measure, verices count() is my 'infinity'
        return len(graph.vs)
    return M_in / 2 / M_out

In [ ]:
def GetResultIndex(modularityResult,quantile):
    maxInd = argrelextrema(np.array(modularityResult), np.greater)
    max_values = np.array(modularityResult)[maxInd]
    
    quantile_value = numpy.quantile(np.array(modularityResult)[maxInd], quantile)
    idx = (np.abs(max_values - quantile_value)).argmin() 

    resultIndex = len(modularityResult)-modularityResult.index(max_values[idx])
    print("Whole algorithm use " + str(len(modularityResult)) + " iterations. Result chosen by local max is from " + str(resultIndex) + " iteration")

    return resultIndex